# Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

# Load Data

In [2]:


# Load the processed 'Adj Close' data from the CSV saved in Task 1
try:
    data = pd.read_csv('../data/adj_close.csv', index_col=0, parse_dates=True)
    # Ensure column names are simplified for easier access if they are still tuples from previous runs
    if isinstance(data.columns, pd.MultiIndex):
        new_cols = []
        for col_tuple in data.columns:
            if 'TSLA' in col_tuple:
                new_cols.append('TSLA')
            elif 'BND' in col_tuple:
                new_cols.append('BND')
            elif 'SPY' in col_tuple:
                new_cols.append('SPY')
            else:
                new_cols.append('_'.join(map(str, col_tuple)).replace('UNNAMED: ', '').replace('_LEVEL_2', ''))
        data.columns = new_cols
        data = data[['TSLA', 'BND', 'SPY']]
    print("Data loaded successfully from adj_close.csv!")
    print("Columns in loaded data:", data.columns)
except Exception as e:
    print(f"Error loading data from adj_close.csv: {e}")
    try:
        raw_data = pd.read_csv('../data/financial_data.csv', header=[0, 1], index_col=0, parse_dates=True)
        data = raw_data['Adj Close']
        data.columns = ['BND', 'SPY', 'TSLA']
        print("Successfully re-extracted 'Adj Close' from 'financial_data.csv'.")
    except Exception as e_fallback:
        print(f"Fallback failed: {e_fallback}. Cannot proceed without proper data.")
        exit()


Data loaded successfully from adj_close.csv!
Columns in loaded data: Index(['('CLOSE', 'BND', 'UNNAMED: 1_LEVEL_2')',
       '('CLOSE', 'SPY', 'UNNAMED: 2_LEVEL_2')',
       '('CLOSE', 'TSLA', 'UNNAMED: 3_LEVEL_2')'],
      dtype='object')


# Calculate Daily Returns

In [ ]:
# --- 1. Calculate Daily Returns ---
# Calculate daily percentage change for all assets
daily_returns = data.pct_change().dropna()
daily_returns.columns = ['BND', 'SPY', 'TSLA']  # Rename columns for consistency

# --- 2. Define Expected Returns ---
tsla_predicted_annual_return = -0.54 # This is a placeholder derived from the visual forecast
tsla_expected_daily_return = (1 + tsla_predicted_annual_return)**(1/252) - 1


# Calculate historical average daily returns (annualized) for BND and SPY
bnd_expected_daily_return = daily_returns['BND'].mean()
spy_expected_daily_return = daily_returns['SPY'].mean()

# Annualize the daily returns for BND and SPY
bnd_annual_return = bnd_expected_daily_return * 252
spy_annual_return = spy_expected_daily_return * 252

expected_returns = np.array([tsla_predicted_annual_return, bnd_annual_return, spy_annual_return])
assets = ['TSLA', 'BND', 'SPY'] # Ensure order matches expected_returns

print("\nExpected Annual Returns (for optimization):")
for i, asset in enumerate(assets):
    print(f"{asset}: {expected_returns[i]:.4f}")


Expected Annual Returns (for optimization):
TSLA: -0.5400
BND: 0.0196
SPY: 0.1448


# Covarianc Matrix

In [ ]:
# --- 3. Covariance Matrix ---
cov_matrix_daily = daily_returns.cov()

# Annualize the covariance matrix
cov_matrix_annual = cov_matrix_daily * 252

print("\nAnnualized Covariance Matrix:")
print(cov_matrix_annual)


Annualized Covariance Matrix:
           BND       SPY      TSLA
BND   0.003016  0.001150  0.001861
SPY   0.001150  0.033272  0.052950
TSLA  0.001861  0.052950  0.350331
